In [199]:
import pandas as pd
import numpy as np

import sys, pathlib, fitz

from dateutil import parser

### Data Import

In [200]:
mint_transaction_history = pd.read_csv('sourceData/mint_2019_2024.csv')

mint_transaction_history.head(10)

mint_transaction_history['Account Name'].unique()

mint_account_aliases = {
    'TD BEYOND CHECKING': 'Checking [3773]',
    'TD CONVENIENCE CHECKING': 'Checking [7985]',
    'HYSA': 'American Express Savings',
    'Individual ...959': 'Charles Schwab Individual Investment Account',
    'CREDITCARD Account': 'Capital One Credit Card'
}

mint_transaction_history['Account Name'].replace(mint_account_aliases, inplace=True)

mint_transaction_history

/var/folders/wz/lh1qtr6d08xgdwbp27c1w9000000gn/T/ipykernel_22126/2634040784.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mint_transaction_history['Account Name'].replace(mint_account_aliases, inplace=True)


,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
0,12/30/2023,LinkedIn,LINKEDIN PRE 975246241,42.49,debit,Career,Apple Card,NaN,NaN
1,12/29/2023,VERIS INSIGHTS L PAYROLL,VERIS INSIGHTS L PAYROLL,2315.42,credit,Paycheck,Checking [3773],NaN,NaN
2,12/29/2023,Blue Bottle Coffee,SQ *BLUE BOTTLE COFF,11.24,debit,Coffee Shops,Checking [3773],NaN,NaN
3,12/29/2023,VERIS INSIGHTS L PAYROLL,VERIS INSIGHTS L PAYROLL,408.60,credit,Paycheck,American Express Savings,NaN,NaN
4,12/29/2023,MBTA KENMORE,MBTA KENMORE,20.00,debit,Public Transportation,Apple Card,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7346,8/10/2018,Starbucks,469216 STARBUCKS STORE 00833469216 STA...,3.37,debit,Coffee Shops,Checking [3773],NaN,NaN
7347,8/10/2018,Sweetgreen Seaport Squ,469216 SWEETGREEN SEAPORT SQ469216 SWE...,13.32,debit,Restaurants,Checking [3773],NaN,NaN
7348,8/09/2018,Starbucks,469216 STARBUCKS STORE 00875469216 STA...,3.47,debit,Coffee Shops,Checking [3773],NaN,NaN
7349,8/08/2018,Starbucks,469216 STARBUCKS STORE 00875469216 STA...,2.94,debit,Coffee Shops,Checking [3773],NaN,NaN


### PDF (Original Balance) Handling

In [201]:
def find_balance(statement_path):
    
    full_doc = fitz.open(statement_path)
    cover_page = full_doc[0]
    words = cover_page.get_text("words", sort=True)

    for i, word_obj in enumerate(words):

        text = word_obj[4]

        if text == 'Ending':

            if words[i + 1][4] == 'Balance':

                end_bal = words[i + 2][4]

                print(text)
                print(words[i + 1][4])
                print(words[i + 2][4])

        if text == 'Statement':

            if words[i + 1][4] == 'Period:':

                end_month = words[i + 4][4].split('-')[1]
                end_day = words[i + 5][4]
                end_year = words[i + 6][4]

                full_statement_end_str = end_month + "-" + end_day + "-" + end_year

                print(text)
                print(words[i + 1][4])
                print(f"{end_month} {end_day} {end_year}")

    return end_bal, full_statement_end_str

original_balance_statement_paths = ['sourceData/OriginalBalance_PDF_9_2018_3773.pdf', 'sourceData/OriginalBalance_PDF_9_2018_7985.pdf']
original_statement_info = {}

for statement_path in original_balance_statement_paths:

    statement_info = find_balance(statement_path)

    account_id = statement_path.split('.')[0][-4:]

    end_bal = statement_info[0]
    full_statement_end_str = statement_info[1]

    original_statement_info[account_id] = [end_bal, full_statement_end_str]

original_statement_info

Statement
Period:
Sep 07 2018
Ending
Balance
2,439.91
Statement
Period:
Sep 11 2018
Ending
Balance
6,551.34


{'3773': ['2,439.91', 'Sep-07-2018'], '7985': ['6,551.34', 'Sep-11-2018']}

### CSV (Mint Transaction History) Handling

In [202]:
mint_transaction_history['Date'] = pd.to_datetime(mint_transaction_history['Date'], format='mixed', dayfirst=False, yearfirst=False)

In [203]:
accounts_end_dates = {str(key): value[1] for key, value in original_statement_info.items()}

for account in accounts_end_dates:

    print(account)
    print(accounts_end_dates[account])

    mint_transaction_history = mint_transaction_history[((mint_transaction_history['Date'] > accounts_end_dates[account]) & (mint_transaction_history['Account Name'].str.contains(account))) | (~mint_transaction_history['Account Name'].str.contains(account))]

original_balance = sum([float(value[0].replace(',', '')) for key, value in original_statement_info.items()])

original_balance

3773
Sep-07-2018
7985
Sep-11-2018


8991.25

In [204]:
mint_transaction_history.loc[mint_transaction_history['Transaction Type'] == 'debit', 'Amount'] *= -1

account_3773 = mint_transaction_history[mint_transaction_history['Account Name'].str.contains('3773')]

initial_bal = float(original_statement_info['3773'][0].replace(',', ''))

account_3773 = account_3773.sort_values('Date', ascending=True)

# account_3773.drop_duplicates(subset=['Date', 'Amount'], inplace=True)

# account_3773 = account_3773[account_3773['Original Description'] != 'VERIS INSIGHTS L']

account_3773['Cumulated Sum'] = initial_bal + account_3773['Amount'].cumsum()

account_3773

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes,Cumulated Sum
7270,2018-09-10,Salem Garage Ma,444500 SALEM GARAGE444500 SALEM GARAGE...,-5.00,debit,Parking,Checking [3773],NaN,NaN,2434.91
7257,2018-09-10,Starbucks,469216 STARBUCKS STORE 08873469216 STA...,-4.76,debit,Coffee Shops,Checking [3773],NaN,NaN,2430.15
7258,2018-09-10,Shaw's,314117 SHAWS 247314117 SHA...,-54.44,debit,Groceries,Checking [3773],NaN,NaN,2375.71
7259,2018-09-10,Starbucks,469216 STARBUCKS STORE 07535469216 STA...,-3.70,debit,Coffee Shops,Checking [3773],NaN,NaN,2372.01
7260,2018-09-10,Land N Sea,426979 LAND N SEA426979 LAND N SEA ...,-13.86,debit,Auto & Transport,Checking [3773],NaN,NaN,2358.15
...,...,...,...,...,...,...,...,...,...,...
31,2023-12-26,420429 SPOTIFY USA,420429 SPOTIFY USA,-10.99,debit,Music,Checking [3773],NaN,NaN,-3595.56
30,2023-12-26,AllisonHouse - AWIPS CAVE Subscription,443565 ALLISONHOUSE LLC,-19.99,debit,Weather Software,Checking [3773],NaN,NaN,-3615.55
21,2023-12-27,APPLECARD GSBANK PAYMENT,APPLECARD GSBANK PAYMENT,-1983.59,debit,Coffee Shops,Checking [3773],NaN,NaN,-5599.14
2,2023-12-29,Blue Bottle Coffee,SQ *BLUE BOTTLE COFF,-11.24,debit,Coffee Shops,Checking [3773],NaN,NaN,-5610.38
